# Tagup Data Science Exercise

ExampleCo, Inc is gathering several types of data for its fleet of very expensive machines.  These very expensive machines have three operating modes: *normal*, *faulty* and *failed*.   The machines run all the time, and usually they are in normal mode.  However, in the event that the machine enters faulty mode, the company would like to be aware of this as soon as possible.  This way they can take preventative action to avoid entering failed mode and hopefully save themselves lots of money.

They collect four kinds of timeseries data for each machine in their fleet of very expensive machines.  When a machine is operating in *normal* mode the data behaves in a fairly predictable way, but with a moderate amount of noise.  Before a machine fails it will ramp into *faulty* mode, during which the data appears visibly quite different.  Finally, when a machine fails it enters a third, and distinctly different, *failed* mode where all signals are very close to 0.

You can download the data here: [exampleco_data](https://drive.google.com/open?id=1b12u6rzkG1AxB6wLGl7IBVoaoSoZLHNR)

__Your main objective: to develop an automated method to pinpoint the times of fault and failure in this machine__.  Keep in mind that you will be sharing these results with the executives at ExampleCo, so to the best of your ability, try to explain what you are doing, what you've shown, and why you think your predictions are good.


A few notes to help:
1. A good place to start is by addressing the noise due to communication
   errors.
2. Feel free to use any libraries you like, or even other programming
   languages. Your final results should be presented in this notebook, however.
3. There are no constraints on the techniques you bring to bear, we are curious
   to see how you think and what sort of resources you have in your toolbox.
4. Be sure to clearly articulate what you did, why you did it, and how the
   results should be interpreted. In particular you should be aware of the
   limitations of whatever approach or approaches you take.
5. Don't feel compelled to use all the data if you're not sure how. Feel free
   to focus on data from a single unit if that makes it easier to get started.
6. Don't hesitate to reach out to datasciencejobs@tagup.io with any questions!

In [ ]:
# Imports
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import signal
import numpy as np
%matplotlib inline 

<b1> To get started, let's take a look at the entire example data to see consistent behaviors from machine to machine. </b1>

In [ ]:
for ii in range(0,10):
    data = pd.read_csv('exampleco_data/machine_'+str(ii)+'.csv',index_col=0)
    plt.plot(range(len(data)), data)
    plt.title('machine_' + str(ii))
    plt.show()

<b1> With an understanding of the entire set of raw data, let's apply the full thought process of detecting machine states to one set of data (machine 0). At the end, the same methodology will be applied to all data to test for correctness. </b1>

<b1> The first step is to remove the noise present in the system. The simplest and most efficient method to remove the numerous peaks is to remove all entries above and below certain thresholds. Here, it's easy to see those thresholds should fall within the +- 150-200 range. These removed values are then replaced with the last present value in the column prior to the removed entry, which retains more of the machine behavior than filling values with the average or median. This is also reliable when gathering data in real time. </b1>

In [ ]:
data = pd.read_csv('exampleco_data/machine_0.csv',index_col=0)
data_low_clip = data[data > -200]
data_clipped = data_low_clip[data_low_clip < 200].fillna(method='ffill')
data_clipped.fillna(method='bfill',inplace=True) #This prevents NaNs from appearing as the first data entry


plt.plot(range(len(data_clipped)), data_clipped)
plt.title('machine_0')
plt.show()

<b1> With the peaks removed from the data, it's much easier to see the behavior of each data stream coming from the machine. Take the first 450 points, where each operating states can be observed: </b1>

In [ ]:
data_clipped[:450].plot(use_index=False, subplots=True, sharex=False, figsize=(12,10))
plt.show()

<b1> However, the data is still noisy, so filtering should be applied to smooth it out. Here, a second-order low pass butterworth filter is used. Since each data stream samples every eight hours approximately, the sampling frequency for the filter is set to once every 8 hours, or 3.472e-5 Hz. This is the reason for choosing such a small Wn. Applying the filter to the machine data will be important, so it is stored as a function for later use. </b1>

In [ ]:
def apply_filter(data_clipped):
    b, a = signal.butter(N=2, Wn=0.000004, btype='low', analog=False, fs=0.00003472)
    data_filtered = pd.DataFrame()
    
    for ii in range(0,4):
        data_filtered[str(ii)] = pd.Series(signal.filtfilt(b,a,data_clipped[str(ii)]))
        
    return data_filtered

In [ ]:
data_filtered = apply_filter(data_clipped)
data_filtered[:450].plot(subplots=True, sharex=False, figsize=(12,10))

<b1> It can now be seen where the behavior of the machine shifts from normal, to faulty, to failed. Normal operation manifests as a regular sine wave (with some small noise artifacts still present). Faulty operation is clearly seen as an increase in the frequency from normal operation of the data streams 0, 1, and 2. The peaks in stream 3 also indicate faulty operation, but the state can be detected much earlier with the other streams. Failed operation is where the signal peaks reduce to nearly zero, as stated in the problem definition. </b1>

<b1> Now that the data is processed and workable, work can be done to detect state changes. As mentioned above, the entrance to a faulty operating state is marked by an increase in signal frequency. By tracking the frequency of the signal, an alert can be sent once this change in frequency occurs. </b1>

In [ ]:
def fault_check(data_filtered):
    faults = pd.DataFrame()
    
    # Comparing adjacent slopes in the data finds where the slope changes from positive to negative.
    # Knowing how frequently the slope changes sign provides the frequency of the wave.
    # The following process is applied to each of the first three data streams deemed relevant (see above).
    for ii in range(0,3):
        
        # Take the gradient of the signal, then create another copy of the gradient and offset it by 1
        #     5.0  4.6  3.2  1.1 -1.2
        # ->  6.3  5.0  4.6  3.2  1.1 
        diff = data_filtered[str(ii)].diff().fillna(0)
        diff2 = data_filtered[str(ii)].diff().shift(1).fillna(0)
        
        # Find the indexes of the data where the sign of the gradient changes 
        #   (signal flips from positive to negative)
        idx = np.where(np.sign(diff.values) != np.sign(diff2.values))[0]
        fault_list = []
        
        # Taking the interval between indexes of sign change gives the signal frequency.
        # Average two adjacent intervals (to inhibit false positives caused by single random short intervals),
        #    and if the result falls below a threshold seen in the data, append to a list of indeces where
        #    the machine is believed to be faulty.
        for kk in range(2,len(idx)):
            first_interval = idx[kk-1] - idx[kk-2]
            second_interval = idx[kk] - idx[kk-1]
            if (second_interval + first_interval) / 2 < 7:
                fault_list.append(idx[kk])
                
        faults[str(ii)] = pd.Series(fault_list)
    
    # A voting function is invoked so that the machine doesn't report a faulty state
    #   unless 2 streams report it for the same index.
    # The following returns the first index that 2 streams deem the machine to be faulty in.
    for jj in range(0,len(data_filtered)):
        vote_count = 0
        if jj in faults['0'].values:
            vote_count += 1
        if jj in faults['1'].values:
            vote_count += 1
        if jj in faults['2'].values:
            vote_count += 1
        if vote_count >= 2:
            return faults, jj
        
    return faults, -1

In [ ]:
faults, fault_idx = fault_check(data_filtered)
print('Voted fault index is: %d' % (fault_idx))
print('Entered faulty operation state on:', data.index[fault_idx])

<b1> With a function for detecting a machine's faulty operation state, there also needs to be a similar function for the detection of a failure state. The failure state detection is much simpler, as the signal frequency isn't necessary. Here, the actual value of the gradient (or slope) of the signal is sufficient. Applying another voting function to the streams and detecting when the slope of the streams falls below another threshold provides a reliable failure state detection function. </b1>

In [ ]:
def failure_check(data_filtered):
    failures = pd.DataFrame()
    
    # Unlike before, all streams are helpful for determining when the machine enters the failure state,
    #    so all streams are iterated through.
    for ii in range(0,4):
        
        # The gradient of the signal is required again
        diff_val = data_filtered[str(ii)].diff().fillna(0).values
        failure_list = []
        
        # Taking the average of two instantaneous slopes (to avoid false positives again)
        #    and comparing to the threshold yields all indices where the stream shows the
        #    machine to be in a failed state
        for kk in range(2,len(diff_val)):
            if (abs(diff_val[kk-1]) + abs(diff_val[kk])) / 2 < 0.25:
                failure_list.append(kk)
        
        failures[str(ii)] = pd.Series(failure_list)
    
    # A voting function is invoked so that the machine doesn't report a failed state 
    #   unless 3 streams report it for the same index.
    # The following returns the first index that 3 streams deem the machine to be failed.
    for jj in range(0,len(data_filtered)):
        vote_count = 0
        if jj in failures['0'].values:
            vote_count += 1
        if jj in failures['1'].values:
            vote_count += 1
        if jj in failures['2'].values:
            vote_count += 1
        if jj in failures['3'].values:
            vote_count += 1
        if vote_count >= 3:
            return failures, jj

    return failures, jj

In [ ]:
failures, fail_idx = failure_check(data_filtered)
print('Voted failed index is: %d' % (fail_idx))
print('Entered failed operation state on:', data.index[fail_idx])

<b1> Now that all the separate pieces are in place, the entire process is fit into one function that reads the data for each machine and determines the fault and failure points for that machine. The filtered data is then plotted with the fault and failure operating locations marked. </b1>

In [ ]:
def automated_fault_failure_detection(address,num_machines):
    for m_num in range(0,num_machines):
        print("For Machine %d" % (m_num))
        data = pd.read_csv(address+'machine_'+str(m_num)+'.csv',index_col=0)
        
        # Clipping out the large noise spikes and replacing missing values
        data_low_clip = data[data > -200]
        data_clipped = data_low_clip[data_low_clip < 200].fillna(method='ffill')
        data_clipped.fillna(method='bfill',inplace=True)
        
        # Applying the low pass filter to smooth the data
        data_filtered = apply_filter(data_clipped)
        
        # Determine the first fault-state and failed-state indices
        faults, fault_idx = fault_check(data_filtered)
        failures, fail_idx = failure_check(data_filtered)
        
        # Plot the results
        plt.figure(figsize=(12,8))
        plt.plot(range(len(data_filtered)), data_filtered)
        
        if fail_idx <= fault_idx:
            print('Machine transitioned directly from normal state to failure state')
        else:
            print('Entered faulty operation state on:', data.index[fault_idx])
            plt.text(fault_idx+10,int(data_filtered.values.max()+8),'faulty')
            plt.plot([fault_idx,fault_idx],[0,int(data_filtered.values.max()+10)],color='black',linewidth=2)
        
        print('Entered failed operation state on:', data.index[fail_idx])
        plt.text(fail_idx+10,5,'failure')
        plt.plot([fail_idx,fail_idx],[0,8],color='black',linewidth=2)
        
        plt.show()

automated_fault_failure_detection('exampleco_data/',10)        

<b1> Each machine is able to be diagnosed when it entered a faulty or failed operating state. When the faulty state check function provides an index/time that is larger or later than the result from the failed state check function, there simply wasn't a faulty state. In these cases, the machine jumped directly from normal operation to failed. The above methods for automatically detecting aren't dependent on having the entire csv full of machine data and can be implemented in a real-time data collection scenario. It can also be seen that the faulty and failed states for each machine were accurately detected and recorded. </b1>

### Thank you!

<b1> Thank you for sending me this exercise. It was a lot of fun! It was enjoyable to think on how I could go about solving it. If you are able, I would love to get feedback of any kind on this. </b1>

<b1> Have a great day! </b1>

<b1> - John A </b1>